## Import

In [ ]:
# 24.02.24
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten

### Read dataframe

In [ ]:
train = pd.read_csv(r"set/train.csv", delimiter=",")
test = pd.read_csv(r"set/test.csv", delimiter=",")
sample_sub = pd.read_csv("set/sample_solution.csv", index_col="Id")

### Reshape

In [ ]:
X_train = np.array(train.iloc[:, 2:]).reshape(-1, 28, 28, 1).astype(np.float32) / 255.0
X_test = np.array(test.iloc[:, 1:]).reshape(-1, 28, 28, 1).astype(np.float32) / 255.0
y_train = train['Categoria']

In [ ]:
y = pd.get_dummies(y_train)
y_col = y.columns  
y = y.to_numpy()

### Create first model

In [ ]:
model = tf.keras.Sequential()

# # First layer, which has a 2D Convolutional layer with kernel size as 3x3 and Max pooling operation
model.add(Conv2D(32, (3,3), padding='same', input_shape=(28,28, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))

# Second layer, which has a 2D Convolutional layer with kernel size as 3x3 & ReLU activation and Max pooling operation
model.add(Conv2D(64, (3,3), padding='same', activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.3))

# Fully connected layer with ReLU activation function 
model.add(Flatten())
model.add(Dense(128, activation=tf.nn.relu))

# Output layer with softmax activation function
model.add(Dense(10, activation=tf.nn.softmax))

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
# model.summary()
# visualkeras.layered_view(model)
# plot_model(model, to_file='model.png')

In [ ]:
train_model = model.fit(X_train, y, epochs=75, batch_size=128, validation_split=0.1, verbose=1)

### Save first prediction

In [ ]:
prediction = model.predict(X_test)
pred_df = pd.DataFrame(prediction,columns=y_col,index=sample_sub.index)
prediction_col = pred_df.idxmax(axis=1)
sample_sub.Categoria = prediction_col

sample_sub.to_csv("submission_1.csv",index=True)


## 6. Tuning the Model


In [ ]:
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

### Create second model

In [ ]:
# Creating a Sequential model
model = tf.keras.Sequential()

# First Convolutional layer:
# - He normal initialization: An initialization technique to set initial weights.
model.add(tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', kernel_initializer='he_normal', input_shape=(28,28, 1)))

# - Reduces the spatial dimensions of the input volume.
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional layer:
# - 64 filters: Captures 64 different features in the input volume.
# - Kernel size 3x3: Each filter scans through 3x3 patches of the input volume.
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))

# - Pool size 2x2: Reduces the dimensionality by taking the maximum value within each 2x2 patch.
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))


# - Dropout rate 0.3: 30% of the input units are randomly dropped during training.
model.add(tf.keras.layers.Dropout(0.3))


# - Normalizes the activations of the previous layer at each batch.
model.add(tf.keras.layers.BatchNormalization())

# - 128 filters: Captures 128 different features in the input volume.
# - Kernel size 3x3: Each filter scans through 3x3 patches of the input volume.
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))

# Max pooling layer:
# - Reduces the spatial dimensions of the input volume.
# - Pool size 2x2: Reduces the dimensionality by taking the maximum value within each 2x2 patch.
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# - Helps prevent overfitting by randomly setting a fraction of input units to zero.
model.add(tf.keras.layers.Dropout(0.4))


# - Flattens the input volume into a 1D array to be fed into a fully connected layer.
model.add(tf.keras.layers.Flatten())

# - Normalizes the activations of the previous layer at each batch.
model.add(tf.keras.layers.BatchNormalization())

# Fully connected Dense layer:
# - 512 units: Each unit represents a weight that will be learned during training.
model.add(tf.keras.layers.Dense(512, activation='relu'))

# Dropout layer:
# - Helps prevent overfitting by randomly setting a fraction of input units to zero.
model.add(tf.keras.layers.Dropout(0.25))

# Output Dense layer
model.add(tf.keras.layers.Dense(10, activation='softmax'))

### Optimize it

In [ ]:
optimizer_2 = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=optimizer_2, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
train_model = model.fit(X_train, y, epochs=75, batch_size=128, validation_split=0.1, verbose=1)

### Save second prediction

In [ ]:
prediction = model.predict(X_test)
pred_df = pd.DataFrame(prediction,columns=y_col,index=sample_sub.index)
prediction_col = pred_df.idxmax(axis=1)
sample_sub.Categoria = prediction_col

sample_sub.to_csv("submission_2.csv",index=True)